# Repräsentation

<img src="pictures/5-Schichten-Architektur.png" alt="5-Schichten-Architektur" width="500" style="background-color: white;"/>

<img src="pictures/Datenelemente-meme.png" alt="Datenelemente-meme" width="500" style="background-color: white;"/>

<img src="pictures/Strings-meme.png" alt="Strings-meme" width="500" style="background-color: white;"/>

<img src="pictures/Versatz-zur-Effizienz.png" alt="Versatz-zur-Effizienz" width="500" style="background-color: white;"/>

<img src="pictures/Speicherung_Metadaten.png" alt="Speicherung_Metadaten" width="500" style="background-color: white;"/>

<img src="pictures/Aufteilung_in_Blöcke.png" alt="Aufteilung_in_Blöcke" width="500" style="background-color: white;"/>

<img src="pictures/Logische-Adressen.png" alt="Logische-Adressen" width="500" style="background-color: white;"/>

<img src="pictures/Hybride-Adressierung.png" alt="Hybride-Adressierung" width="500" style="background-color: white;"/>

<img src="pictures/Finden-von-Fehlern.png" alt="Finden-von-Fehlern" width="500" style="background-color: white;"/>

<img src="pictures/Datensätze-variabler-Länge.png" alt="Datensätze-variabler-Länge" width="500" style="background-color: white;"/>

<img src="pictures/Linked-open-data.png" alt="Linked-open-data" width="500" style="background-color: white;"/>

<img src="pictures/SPARSE-columns.png" alt="SPARSE-columns" width="500" style="background-color: white;"/>

<img src="pictures/Einfügen-mit-Platz.png" alt="Einfügen-mit-Platz" width="500" style="background-color: white;"/>

<img src="pictures/Löschen-meme.png" alt="Löschen-meme" width="500" style="background-color: white;"/>

<img src="pictures/update.png" alt="update" width="500" style="background-color: white;"/>